In [122]:
import bf
from math import log2, ceil

def idx2inp(idx, bitcount):
  return [int(c) for c in str(bin(idx))[2:].zfill(bitcount)]

def idx2inp2(idx, bitcount):
  incidence = [0]*bitcount
  for i in range(bitcount):
    div = 2**(bitcount-i-1)
    if div <= idx:
      idx = idx % div 
      incidence[i] = 1
  return incidence

%timeit idx2inp
%timeit idx2inp2


20.7 ns ± 1.7 ns per loop (mean ± std. dev. of 7 runs, 100000000 loops each)
23.9 ns ± 2.15 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [2]:
import cgm 
import bf 
import blame
from random import randint

f = bf.BF([randint(0,1) for _ in range(8)])
# f = bf.BF([0, 1, 1, 0, 1, 1, 1, 1]) -> interesting!
print(f)
print("-"*20)
banzhaf_sym_omega = lambda i,f: cgm.banzhaf(cgm.sym_omega, i, f)
banzhaf_sym_nu = lambda i,f: cgm.banzhaf(cgm.sym_nu, i, f)
shapley_sym_omega = lambda i,f: cgm.shapley(cgm.sym_omega, i, f)
shapley_sym_nu = lambda i,f: cgm.shapley(cgm.sym_nu, i, f)
rank_blame = bf.ranking(blame.blame, f)
rank_banz_omega = bf.ranking(banzhaf_sym_omega, f)
rank_banz_nu = bf.ranking(banzhaf_sym_nu, f)
rank_shap_omega = bf.ranking(shapley_sym_omega, f)
rank_shap_nu = bf.ranking(shapley_sym_nu, f)

print(bf.ranking2str(rank_blame))
print(bf.ranking2str(rank_banz_omega))
print(bf.ranking2str(rank_banz_nu))
print(bf.ranking2str(rank_shap_omega))
print(bf.ranking2str(rank_shap_nu))

  x0    x1    x2    f(x)
----  ----  ----  ------
   0     0     0       1
   0     0     1       0
   0     1     0       1
   0     1     1       0
   1     0     0       0
   1     0     1       0
   1     1     0       0
   1     1     1       1
--------------------
x1:0.625 < x2:0.875 == x0:0.875
x1: 0.0 < x2:0.0156 == x0:0.0156
x2:0.0156 == x0:0.0156 == x1:0.0156
x1:   0 < x2: 0.5 == x0: 0.5
x2:0.3333 == x0:0.3333 == x1:0.3333


False


In [18]:
import bf
import cgm
import blame

def is_np(v, f, x):
  for S in bf.itersets(f.bitcount):
    if x not in S and v(S.union({x}), f) != v(S, f):
      return False
  return True

def is_independent(f, x):
  for u in bf.iterinp(f.bitcount):
    if f[ bf.flip(u, {x}) ] != f[u]:
      return False
  return True

for f in bf.iterfunc(4):
  v = cgm.sym(cgm.nu)
  stop = False
  for x in range(f.bitcount):
    if is_np(v, f, x) and not is_independent(f,x):
      print(f"f not independent of x{x}, but its a null variable: f = {bf.sum_notation( f.prime_implicants() )}")
      print(bf.ranking2str(bf.ranking(blame.blame, f)), "Blame")
      print(bf.ranking2str(bf.ranking(cgm.get_value("Bz/Sym/Rec"), f)), "Bz/Sym/Rec")
      stop = True
      break
  if stop:
    break

f not independent of x3, but its a null variable: f = x0x1'x2 + x0x2x3 + x0x1x2' + x0x1x3
x3:0.2969 < x2:0.6302 == x1:0.6302 < x0:0.7188 Blame
x3:0.0000 < x1:0.2500 == x2:0.2500 < x0:0.7500 Bz/Sym/Rec


In [6]:
from axioms import get_symmetric_bits
import bf
bitcount = 4
i,j = 0,1
nr_sym_functions = 0
for idx in range(2**(2**bitcount)):
  f = bf.BF(bf.idx2inp(idx, 2**bitcount))
  syms = get_symmetric_bits(f)
  if (i,j) in syms or (j,i) in syms:
    nr_sym_functions += 1
print(nr_sym_functions)

4096


In [1]:
import bf 
import blame 

f = bf.BF([0,0,1,1])
rank = bf.ranking(blame.blame, f)
print(bf.ranking2str(rank))

x1: 0.0 < x0: 1.0


In [3]:
import cgm 
import bf

# prints all functions where the first bit is (a) a dummy in nu but (b) not a dummy in f
bits = 3
for idx in range(2**(2**bits)):
  f = bf.BF( bf.idx2inp(idx, 2**bits) )
  is_dummy_in_f = True 
  for uidx in range(2**bits):
    u = bf.idx2inp(uidx, bits)
    if f[ bf.flip(u, { 0 }) ] != f[u]:
      is_dummy_in_f = False 
      break
  crit = list(cgm.crits(cgm.nu, 0, f))
  if len(crit) == 0 and not is_dummy_in_f:
    print(f)

[]
  x0    x1    x2    f(x)
----  ----  ----  ------
   0     0     0       0
   0     0     1       1
   0     1     0       1
   0     1     1       0
   1     0     0       0
   1     0     1       1
   1     1     0       1
   1     1     1       1
[]
  x0    x1    x2    f(x)
----  ----  ----  ------
   0     0     0       0
   0     0     1       1
   0     1     0       1
   0     1     1       0
   1     0     0       1
   1     0     1       1
   1     1     0       1
   1     1     1       0
[]
  x0    x1    x2    f(x)
----  ----  ----  ------
   0     0     0       0
   0     0     1       1
   0     1     0       1
   0     1     1       1
   1     0     0       0
   1     0     1       1
   1     1     0       1
   1     1     1       0
[]
  x0    x1    x2    f(x)
----  ----  ----  ------
   0     0     0       1
   0     0     1       0
   0     1     0       0
   0     1     1       1
   1     0     0       1
   1     0     1       0
   1     1     0       1
   1     1   

In [7]:
import bf

# how 2 convert lambda function to list
lf = lambda x0, x1: x0 and not x1
bits = 2
f = [0]*(2**bits)
for u in bf.iterinp(bits):
    idx = bf.inp2idx(u)
    f[idx] = int(lf(*u))
f = bf.BF(f)
print(f)

  x0    x1    f(x)
----  ----  ------
   0     0       0
   0     1       0
   1     0       1
   1     1       0


In [4]:
import blame
import cgm
import bf
from tabulate import tabulate

f = bf.from_lambda(lambda x0,x1,x2,x3,x4: (x0 and x1) or (x2 and x3 and x4))
# valx0 = cgm.banzhaf(cgm.omega, 0, f)
# valx2 = cgm.banzhaf(cgm.omega, 2, f)
# print(valx0, valx2)

crits = list( cgm.crits(cgm.omega, 0, f) )
for c in crits:
    print(c)
# table, headers = cgm.cgm_table(f, cgm.sym_omega)
# print(tabulate(table, headers))

[1]
[2, 1]
[3, 1]
[3, 2, 1]
[4, 1]
[4, 2, 1]
[4, 3, 1]
